In [1]:
import wbdata
import pandas as pd
import datetime
import requests
import matplotlib.pyplot as plt
import numpy as np
import statsmodels.api as sm
from mpl_toolkits.mplot3d import Axes3D

In [2]:
#Colection of data from World Bank
data_date = (datetime.datetime(2007, 1, 1), datetime.datetime(2024, 12, 31))

indicators = {'NY.GDP.MKTP.CD' : 'GDP_USD'}

data = wbdata.get_dataframe(indicators, country='KEN')

print(data.head())

           GDP_USD
date              
2024  1.244987e+11
2023  1.080386e+11
2022  1.144490e+11
2021  1.097037e+11
2020  1.006575e+11


In [3]:
#Setting the date to the required time and using the date format
start_date = datetime.datetime(2007, 1, 1)
end_date = datetime.datetime(2024, 1, 1)

gdp_data = data.reset_index()

gdp_data['date'] = pd.to_datetime(gdp_data['date'], format='%Y', errors='coerce')
gdp_data = gdp_data.sort_values('date')

gdp_data = gdp_data[(gdp_data['date'] >= start_date) & (gdp_data['date'] <= end_date)]

print(gdp_data.head())

         date       GDP_USD
17 2007-01-01  3.195820e+10
16 2008-01-01  3.589515e+10
15 2009-01-01  4.234722e+10
14 2010-01-01  4.540562e+10
13 2011-01-01  4.686947e+10


In [4]:
#Drop any missing values
gdp_new_data = gdp_data.dropna()

print(gdp_new_data.head())

         date       GDP_USD
17 2007-01-01  3.195820e+10
16 2008-01-01  3.589515e+10
15 2009-01-01  4.234722e+10
14 2010-01-01  4.540562e+10
13 2011-01-01  4.686947e+10


In [5]:
#Using the exchangerate api to get value of a dollar in kes in realtime
url ="https://open.er-api.com/v6/latest/USD" 
response = requests.get(url)
data = response.json()

usd_to_kes = data["rates"]["KES"]
print(f"The exchange rate used is: {usd_to_kes}")

The exchange rate used is: 129.166284


In [6]:
#Converting the GDP column
gdp_new_data['GDP_KES'] = gdp_new_data['GDP_USD'] * usd_to_kes

print(gdp_new_data.head())

         date       GDP_USD       GDP_KES
17 2007-01-01  3.195820e+10  4.127921e+12
16 2008-01-01  3.589515e+10  4.636444e+12
15 2009-01-01  4.234722e+10  5.469833e+12
14 2010-01-01  4.540562e+10  5.864875e+12
13 2011-01-01  4.686947e+10  6.053956e+12


In [8]:
#Preparing the data
gdp_new_data['Year'] = gdp_new_data['date'].dt.year
gdp_clean_new_data = gdp_new_data[['Year', 'GDP_KES']]

print(gdp_clean_new_data.tail())

   Year       GDP_KES
4  2020  1.300156e+13
3  2021  1.417001e+13
2  2022  1.478295e+13
1  2023  1.395494e+13
0  2024  1.608103e+13


In [18]:
cbk_data = pd.read_csv("C:\DATA ANALYTICS\PYDATA\PyData_Project\data\Mobile_Payments_2010-2024.csv")

<>:1: SyntaxWarning: invalid escape sequence '\D'
<>:1: SyntaxWarning: invalid escape sequence '\D'
C:\Users\HP ELITEBOOK 840 G6\AppData\Local\Temp\ipykernel_27256\3026982200.py:1: SyntaxWarning: invalid escape sequence '\D'
  cbk_data = pd.read_csv("C:\DATA ANALYTICS\PYDATA\PyData_Project\data\Mobile_Payments_2010-2024.csv")


In [19]:
#Setting the data in the desired format
cbk_data['date'] = pd.to_datetime(cbk_data['Year'].astype(str) + '-' + cbk_data['Month'], format='%Y-%B')
cbk_new_data = cbk_data.sort_values('date')
print(cbk_new_data.head())

     Year  Month  Active Agents  Active Agents.1  \
213  2007  March            307         0.020992   
212  2007  April            362         0.054944   
211  2007    May            447         0.107733   
210  2007   June            527         0.175652   
209  2007   July            681         0.268499   

     Total Agent Cash in Cash Out (Volume Million)  \
213                                       0.021714   
212                                       0.070000   
211                                       0.150000   
210                                       0.233661   
209                                       0.354298   

     Total Agent Cash in Cash Out (Value KSh billions)       date  
213                                           0.064391 2007-03-01  
212                                           0.220896 2007-04-01  
211                                           0.483709 2007-05-01  
210                                           0.720102 2007-06-01  
209                   

In [20]:
#Setting the date as the index
Mobile_data = cbk_new_data.set_index('date')
print(Mobile_data.tail())

            Year      Month  Active Agents  Active Agents.1  \
date                                                          
2024-08-01  2024     August         348065            78.64   
2024-09-01  2024  September         367551            79.37   
2024-10-01  2024    October         358621            80.82   
2024-11-01  2024   November         383624            81.55   
2024-12-01  2024   December         381116            82.43   

            Total Agent Cash in Cash Out (Volume Million)  \
date                                                        
2024-08-01                                         214.25   
2024-09-01                                         196.49   
2024-10-01                                         215.26   
2024-11-01                                         297.02   
2024-12-01                                         309.28   

            Total Agent Cash in Cash Out (Value KSh billions)  
date                                                           
20

In [21]:
#Aggregating the CBK mobile data by year(2007-2024)
cbk_yearly = Mobile_data.groupby('Year').agg({
    'Total Agent Cash in Cash Out (Value KSh billions)': 'sum',
   
    'Active Agents': 'mean'
}).reset_index()

print(cbk_yearly.tail())

    Year  Total Agent Cash in Cash Out (Value KSh billions)  Active Agents
13  2020                                           5213.543  251099.666667
14  2021                                           6868.770  298064.250000
15  2022                                           7908.830  306987.500000
16  2023                                           7953.930  328422.833333
17  2024                                           8697.740  345117.666667


In [23]:
#Merging the two datasets
merged_data = pd.merge(cbk_yearly, gdp_new_data, on='Year', how='inner')

print(merged_data.tail())

    Year  Total Agent Cash in Cash Out (Value KSh billions)  Active Agents  \
13  2020                                           5213.543  251099.666667   
14  2021                                           6868.770  298064.250000   
15  2022                                           7908.830  306987.500000   
16  2023                                           7953.930  328422.833333   
17  2024                                           8697.740  345117.666667   

         date       GDP_USD       GDP_KES  
13 2020-01-01  1.006575e+11  1.300156e+13  
14 2021-01-01  1.097037e+11  1.417001e+13  
15 2022-01-01  1.144490e+11  1.478295e+13  
16 2023-01-01  1.080386e+11  1.395494e+13  
17 2024-01-01  1.244987e+11  1.608103e+13  
